In [1]:
from ALS.CP import solve_multiple_runways_cp,solve_single_runway_cp
from ALS.MIP import solve_multiple_runways_mip, solve_single_runway_mip
from ALS.utils import read_data
from ALS.performanceCP import performance_CP
from ALS.performanceMIP import performance_MIP
from ortools.sat.python import cp_model
import pandas as pd 

In [2]:
import ortools
print(ortools.__version__)


9.11.4210


In [3]:
filename = "data/airland3.txt"

num_planes, planes_data, separation_times = read_data(filename)

num_runways = 1

	       Reading data from airland3.txt

Number of planes: 20 



In [ ]:
decision_strategies_multiple = [
    {
        "variables": "position",
        "variable_strategy": cp_model.CHOOSE_FIRST,        
        "value_strategy": cp_model.SELECT_MIN_VALUE      
    },
    {
        "variables": "runway",
        "variable_strategy": cp_model.CHOOSE_FIRST,      
        "value_strategy": cp_model.SELECT_MAX_VALUE      
    }
]


solver, mem_before, mem_after = solve_multiple_runways_cp(num_planes, num_runways, planes_data, separation_times)
metrics_CP_multiple_runway = performance_CP(solver, planes_data, mem_before, mem_after)

In [ ]:
decision_strategies_single = [
    {
        "variables": "position",
        "variable_strategy": cp_model.CHOOSE_FIRST,        
        "value_strategy": cp_model.SELECT_MIN_VALUE      
    }
]

solver, mem_before, mem_after = solve_single_runway_cp(num_planes, planes_data, separation_times, decision_strategies_single)
metrics_CP_single_runway = performance_CP(solver, planes_data, mem_before, mem_after)

In [ ]:
solver, variables, num_planes, memory_before, memory_after = solve_multiple_runways_mip(num_planes, num_runways, planes_data, separation_times)
metrics_MIP_multiple_runway = performance_MIP(solver, mem_before, mem_after)

In [7]:
selected_variables = [
    "metrics_CP_single_runway",
    "metrics_CP_multiple_runway"
]

# Filtrar as variáveis selecionadas
variables = {var: globals()[var] for var in selected_variables}

# Criar o DataFrame
data = {
    "Model": list(variables.keys()),
    "Execution Time": [v[0] for v in variables.values()],
    "Status": [v[1] for v in variables.values()],
    "Memory Usage": [v[2] for v in variables.values()],
    "Number of Conflicts": [v[3] for v in variables.values()],
    "Number of Branches": [v[4] for v in variables.values()],
    "Best Objective Bound": [v[5] for v in variables.values()]
}

df = pd.DataFrame(data)

In [ ]:
df.head()

In [4]:
filename = "data/airland1.txt"

num_planes, planes_data, separation_times = read_data(filename)

solver, variables, num_planes, memory_before, memory_after = solve_single_runway_mip(num_planes, planes_data, separation_times)
metrics_MIP_single_runway = performance_MIP(solver, memory_before, memory_after)

	       Reading data from airland1.txt

Number of planes: 10 

		    Creating MIP Solver

-> Number of decision variables created: 120
-> Number of constraints: 205

			Solving MIP

-> Optimal Cost: 700.0

-> Planes that did not land on the target time:
  -> Plane 0: LandingTime_0 | Target Time: 155 | Penalty: 100.0
  -> Plane 4: LandingTime_4 | Target Time: 123 | Penalty: 150.0
  -> Plane 5: LandingTime_5 | Target Time: 135 | Penalty: 30.0
  -> Plane 6: LandingTime_6 | Target Time: 138 | Penalty: 360.0
  -> Plane 7: LandingTime_7 | Target Time: 140 | Penalty: 60.0

		Performance Metrics for MIP

-> Execution time: 0.10 seconds
-> Number of variables in the model: 120
-> Number of constraints in the model: 205
-> Total penalty: 700.0
-> Memory usage: 6.75 MB

